In [1]:
from mistralai import Mistral
import os
import pandas as pd
from pathlib import Path
from sqlutils import sqlutils
import time

In [2]:
class MistralAPI:
    """
    A client for interacting with the MistralAI API.

    Attributes:
        client (Mistral): The Mistral client instance.
        model (str): The model to use for queries.
    """

    def __init__(self, model: str) -> None:
        """
        Initializes the MistralAPI with the given model.

        Args:
            model (str): The model to use for queries.

        Raises:
            ValueError: If the MISTRAL_API_KEY environment variable is not set.
        """
        api_key = os.getenv("MISTRAL_API_KEY")
        if not api_key:
            raise ValueError(
                "No MISTRAL_API_KEY as environment variable, please set it!"
            )
        self.client = Mistral(api_key=api_key)
        self.model = model

    def query(self, query: str, temperature: float = 0.5) -> str:
        """
        Sends a query to the MistralAI API and returns the response.

        Args:
            query (str): The input query to send to the model.
            temperature (float, optional): The temperature parameter for controlling
                                          the randomness of the output. Defaults to 0.5.

        Returns:
            str: The response from the API.
        """
        chat_response = self.client.chat.complete(
            model=self.model,
            temperature=temperature,
            messages=[
                {
                    "role": "user",
                    "content": query,
                },
            ],
        )
        return chat_response.choices[0].message.content

In [3]:
# Déterminer le chemin du script
script_path = Path().resolve()

# # Select des avis depuis la base de données et depuis le chemin du script
bdd = sqlutils(script_path / "../../data/friands.db")
# Déterminer la date du jour puis la date du jour moins 18 mois
date_min = pd.Timestamp.now() - pd.DateOffset(months=18)
# Convertir la date en YYYY-MM-DD
date_min = date_min.strftime("%Y-%m-%d")
print(f"Date minimum: {date_min}")
# Extraire tous les avis dont la date est supérieure à date_min
query = f"SELECT * FROM avis WHERE date_avis >= '{date_min}'"
success, t_avis = bdd.select(query)

if not success:
    print("Erreur lors de l'extraction des avis depuis la base de données")
    print(t_avis)
else:
    print(
        f"Extraction de {len(t_avis)} enregistrements depuis la base de données réussie"
    )

# # Insérer les champs extraits de la base de données dans un dataframe
df = pd.DataFrame(
    t_avis,
    columns=[
        "id_avis",
        "id_restaurant",
        "nom_utilisateur",
        "note_restaurant",
        "date_avis",
        "titre_avis",
        "contenu_avis",
        "label",
    ],
)

Date minimum: 2023-07-15
Extraction de 415 enregistrements depuis la base de données réussie


In [4]:
# Sélectionner les id_restaurant et les noms des restaurants depuis la table restaurant
success, t_resto = bdd.select("SELECT id_restaurant, nom FROM restaurants")

if not success:
    print(
        "Erreur lors de l'extraction des informations des restaurants depuis la base de données"
    )
    print(t_resto)
else:
    # Ajouter les infos au dataframe df
    df_resto = pd.DataFrame(t_resto, columns=["id_restaurant", "nom_restaurant"])
    df = pd.merge(df, df_resto, on="id_restaurant")

In [5]:
# joindre tous les avis pour un restaurant donné
df_grouped = (
    df.groupby("id_restaurant")
    .agg(
        {
            "nom_restaurant": "first",
            "contenu_avis": lambda x: " --- ".join(x),
        }
    )
    .reset_index()
)

In [6]:
os.environ["MISTRAL_API_KEY"] = "ICTv6vD2bgvtkHgjrwNxxLIL82VSxUs0"

In [7]:
# Lister tous les modèles accessibles depuis l'API Mistral
from mistralai import Mistral
import os

with Mistral(
    api_key=os.getenv("MISTRAL_API_KEY", ""),
) as mistral:

    liste_modeles = mistral.models.list()

    assert liste_modeles is not None

In [8]:
# Lister les modèles contenus dans res.data par ordre de max_context_length
liste_modeles_tri = sorted(
    liste_modeles.data, key=lambda x: x.max_context_length, reverse=True
)

for modele in liste_modeles_tri:
    print(
        f"id du modèle : {modele.id} - nb de caractères max pour l'input : {modele.max_context_length}"
    )

id du modèle : codestral-2501 - nb de caractères max pour l'input : 262144
id du modèle : codestral-latest - nb de caractères max pour l'input : 262144
id du modèle : codestral-2412 - nb de caractères max pour l'input : 262144
id du modèle : codestral-2411-rc5 - nb de caractères max pour l'input : 262144
id du modèle : codestral-mamba-2407 - nb de caractères max pour l'input : 262144
id du modèle : open-codestral-mamba - nb de caractères max pour l'input : 262144
id du modèle : codestral-mamba-latest - nb de caractères max pour l'input : 262144
id du modèle : ministral-3b-2410 - nb de caractères max pour l'input : 131072
id du modèle : ministral-3b-latest - nb de caractères max pour l'input : 131072
id du modèle : ministral-8b-2410 - nb de caractères max pour l'input : 131072
id du modèle : ministral-8b-latest - nb de caractères max pour l'input : 131072
id du modèle : open-mistral-nemo - nb de caractères max pour l'input : 131072
id du modèle : open-mistral-nemo-2407 - nb de caractère

### Choix du modèle

In [9]:
# Mettre un modèle basé sur la liste précédente
model = "pixtral-large-latest"
# model = "codestral-latest"


# Instanciation de la classe MistralAPI


model_mistral = MistralAPI(model=model)


# Définition du nombre de caractère max


model_max_length = [m.max_context_length for m in liste_modeles_tri if m.id == model][0]


# Résumé


print(f"Modèle sélectionné : {model}. Longueur de prompt maximale : {model_max_length}")

Modèle sélectionné : pixtral-large-latest. Longueur de prompt maximale : 131072


In [10]:
def split_text(text, max_length):
    """
    Splits the text into chunks of maximum length.
    """
    words = text.split()
    for i in range(0, len(words), max_length):
        yield " ".join(words[i : i + max_length])


query = "Analyser ces avis de clients concernant un restaurant, puis produire un unique résumé de ces avis, court mais riche d'informations. Ne pas produire de liste de points positifs ou négatifs, ni ."
temperature = 0.1

for index, row in df_grouped.iterrows():
    reviews = row["contenu_avis"]
    chunks = list(
        split_text(reviews, model_max_length)
    )  # Adjust the chunk size as needed
    summaries = []
    for chunk in chunks:
        summary = model_mistral.query(f"{query} : '{chunk}'", temperature=temperature)
        summaries.append(summary)
    full_summary = " ".join(summaries)
    print(f"Restaurant {index}: {row['nom_restaurant']}")
    print(f"Résumé: {full_summary}")
    print("\n")

    # Ajouter le résumé à df_grouped
    df_grouped.loc[index, "resume"] = full_summary

    # Marquer une pause de 10 secondes pour ne pas saturer l'API
    time.sleep(10)

Restaurant 0: KUMA cantine
Résumé: Le restaurant, tenu par un couple charmant, séduit par sa décoration typique et sa cuisine maison, notamment ses gyoza, ramen et desserts au matcha. Le service est rapide et agréable, et les prix sont raisonnables. Cependant, une expérience négative a été rapportée concernant un plat de saumon, jugé de mauvaise qualité et mal préparé, laissant une impression de mépris pour le client. En général, le restaurant est recommandé pour sa qualité et son ambiance, malgré cette déception ponctuelle.


Restaurant 1: Mattsam Restaurant Messob
Résumé: Le restaurant offre une expérience culinaire immersive et mémorable de la cuisine éthiopienne, idéale pour les amateurs de saveurs exotiques ou les curieux. Le cadre, décoré avec des éléments traditionnels, est chaleureux et dépaysant. Le personnel, très accueillant et souriant, guide parfaitement les clients, même ceux qui découvrent cette cuisine pour la première fois. Les plats, souvent servis sur de grandes gale

### Mettre à jour la base de données

In [11]:
# Updater les résumés dans la base de données
for index, row in df_grouped.iterrows():
    success, t_insert = bdd.update(
        table_name="restaurants",
        data={"summary": row["resume"]},
        where=[f"id_restaurant = '{row['id_restaurant']}'"],
    )

    if not success:
        bdd.rollback()
        print(
            f"Erreur lors de l'insertion du résumé pour le restaurant {row['id_restaurant']} : {t_insert}"
        )
        print(t_insert)
    else:
        bdd.commit()
        print(f"Résumé inséré pour le restaurant {row['id_restaurant']} ({t_insert})")

Résumé inséré pour le restaurant 1 (1 row(s) successfully updated)
Résumé inséré pour le restaurant 2 (1 row(s) successfully updated)
Résumé inséré pour le restaurant 3 (1 row(s) successfully updated)
Résumé inséré pour le restaurant 4 (1 row(s) successfully updated)
Résumé inséré pour le restaurant 5 (1 row(s) successfully updated)
Résumé inséré pour le restaurant 6 (1 row(s) successfully updated)
Résumé inséré pour le restaurant 7 (1 row(s) successfully updated)
Résumé inséré pour le restaurant 8 (1 row(s) successfully updated)
Résumé inséré pour le restaurant 9 (1 row(s) successfully updated)
Résumé inséré pour le restaurant 10 (1 row(s) successfully updated)
